In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d uciml/red-wine-quality-cortez-et-al-2009


Dataset URL: https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009
License(s): DbCL-1.0
red-wine-quality-cortez-et-al-2009.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from zipfile import ZipFile
dataset = "/content/red-wine-quality-cortez-et-al-2009.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("Done Extracting")

Done Extracting


# Add all dependency

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score

# Import Dataset

In [5]:
df = pd.read_csv("/content/winequality-red.csv")

In [6]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [8]:
type(df)

pandas.core.frame.DataFrame

In [9]:
df.shape

(1599, 12)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


# Preprocessing

In [11]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [12]:
df.duplicated().sum()

240

In [13]:
df.drop_duplicates(inplace=True)

In [14]:
df.duplicated().sum()

0

In [15]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000
mean,8.310596,0.529478,0.272333,2.523400,0.088124,15.893304,46.825975,0.996709,3.309787,0.658705,10.432315,5.623252
std,1.736990,0.183031,0.195537,1.352314,0.049377,10.447270,33.408946,0.001869,0.155036,0.170667,1.082065,0.823578
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996700,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.430000,2.600000,0.091000,21.000000,63.000000,0.997820,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [16]:
df.groupby("quality").mean()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
3,8.360000,0.884500,0.171000,2.635000,0.122500,11.000000,24.900000,0.997464,3.398000,0.570000,9.955000
4,7.779245,0.693962,0.174151,2.694340,0.090679,12.264151,36.245283,0.996542,3.381509,0.596415,10.265094
5,8.171231,0.578596,0.244523,2.509532,0.093971,17.161179,57.406412,0.997082,3.300815,0.623570,9.888562
6,8.337383,0.496084,0.279364,2.457290,0.085075,15.637383,40.816822,0.996565,3.318000,0.674542,10.658879
7,8.858683,0.403653,0.371856,2.716766,0.076832,13.994012,35.449102,0.996051,3.290060,0.742635,11.491118
8,8.441176,0.427647,0.382941,2.576471,0.068353,13.764706,34.470588,0.995095,3.274118,0.774706,12.158824


In [ ]:
columns_to_process = df.iloc[:,0:11]

for i in columns_to_process:
  plt.figure(figsize=(10,5))
  sns.boxplot(data=df, x=i)

In [ ]:
for column in columns_to_process:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)

print(df)

In [ ]:
for i in columns_to_process:
  plt.figure(figsize=(10,5))
  sns.boxplot(data=df, x=i)

scaling

In [20]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [21]:
x = df.drop("quality",axis=1)

In [ ]:
print(x)

In [23]:
x_scaler = scaler.fit_transform(x)

In [24]:
X = x_scaler

In [25]:
print(X)

[[-5.33308047e-01  9.71575614e-01 -1.39415656e+00 ...  1.34139845e+00
  -6.54980503e-01 -9.61206742e-01]
 [-2.92011069e-01  1.98739775e+00 -1.39415656e+00 ... -7.26282032e-01
   2.18683986e-01 -5.87463325e-01]
 [-2.92011069e-01  1.31018299e+00 -1.18935072e+00 ... -3.26085810e-01
   2.67863775e-04 -5.87463325e-01]
 ...
 [-1.43817172e+00  1.25057165e-01 -8.82141974e-01 ...  1.40809782e+00
   8.01126978e-01  7.20638637e-01]
 [-1.43817172e+00  6.61185516e-01 -7.79739058e-01 ...  1.74159467e+00
   4.37100108e-01 -2.13719907e-01]
 [-1.37784747e+00 -1.22937235e+00  1.01231198e+00 ...  5.41006003e-01
   7.30732378e-02  5.33766929e-01]]


# Feature Engineering

In [34]:
correlation = df.corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(correlation,annot=True, cbar=True, fmt=".1f", cmap="Greens", annot_kws={"size":8}, square=True)
plt.show()

# Data Transformation
Binarization

In [29]:
df["quality"].value_counts()

quality
5    577
6    535
7    167
4     53
8     17
3     10
Name: count, dtype: int64

In [30]:
Y =df["quality"].apply(lambda y_value: 1 if y_value>=6 else 0)

In [31]:
print(Y)

0       0
1       0
2       0
3       1
5       0
       ..
1593    1
1594    0
1595    1
1597    0
1598    1
Name: quality, Length: 1359, dtype: int64


# Balancing Dtaset
Undersampling

In [ ]:
good = df["quality"]=1
bad = df["quality"]=0
good_sample = good.sample(n=500)
new_df = pd.concat(good_sample,bad, axis=0)


# Train-Test Splitting

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [61]:
model = LogisticRegression()

In [62]:
model.fit (X_train, Y_train)

LogisticRegression()

# Model Training
Logistic Regression

Support Vector Machine

Random Forest

# Logistic Regression

In [67]:
logistic_train_predict = model.predict(X_train)

In [68]:
print("Accuracy of train data:", accuracy_score(logistic_train_predict,Y_train)*100)

Accuracy of train data: 74.88500459981601


In [69]:
logistic_test_predict = model.predict(X_test)

In [70]:
print("Accuracy of test data:", accuracy_score(logistic_test_predict,Y_test)*100)

Accuracy of test data: 74.26470588235294


# SUpport Vector Machine

In [73]:
model1 = svm.SVC(kernel="linear")

In [74]:
model1.fit(X_train, Y_train)

SVC(kernel='linear')

In [78]:
vector_train_predict = model1.predict(X_train)
vector_test_predict = model1.predict(X_test)

In [76]:
print("Accuracy of train data:", accuracy_score(vector_train_predict,Y_train)*100)

Accuracy of train data: 74.70101195952162


In [79]:
print("Accuracy of test data:", accuracy_score(vector_test_predict,Y_test)*100)

Accuracy of test data: 75.73529411764706


# Random Forest

In [80]:
model2 = RandomForestClassifier()
model2.fit(X_train, Y_train)

RandomForestClassifier()

In [81]:
random_train_predict = model2.predict(X_train)
random_test_predict = model2.predict(X_test)

In [82]:
print("Accuracy of train data:", accuracy_score(random_train_predict,Y_train)*100)

Accuracy of train data: 100.0


In [83]:
print("Accuracy of test data:", accuracy_score(random_test_predict,Y_test)*100)

Accuracy of test data: 79.41176470588235


# Gradient Boosting

In [84]:
model3 = GradientBoostingClassifier()
model3.fit(X_train, Y_train)

GradientBoostingClassifier()

In [85]:
gradient_train_predict = model3.predict(X_train)
gradient_test_predict = model3.predict(X_test)

In [86]:
print("Accuracy of train data:", accuracy_score(gradient_train_predict,Y_train)*100)

Accuracy of train data: 86.29254829806807


In [88]:
print("Accuracy of train data:", accuracy_score(gradient_test_predict,Y_test)*100)

Accuracy of train data: 76.10294117647058


# Prediction

In [96]:
input = (7.5,0.5,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.8,10.5)

input_as_nparray = np.asarray(input)

input_reshape = input_as_nparray.reshape(1,-1)

prediction = model.predict(input_reshape)
print(prediction)

if (prediction==1):
  print("Wine quality good")
else:
  print("Wine quality bad")

[0]
Wine quality bad
